In [4]:
!pip install openai


[notice] A new release of pip is available: 24.0 -> 24.1.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [5]:
import json
import minsearch

with open("documents.json","r") as docs:
    documents_raw = json.load(docs)

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [6]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    # required but ignored
    api_key='ollama',
)

In [8]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='phi3',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [9]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [13]:
rag("can i run dbt locally")

' Yes, you can run dbt locally using Docker and Postgres as per the provided context. Here are detailed steps based on that information:\n\n1) First, ensure your postgres instance is running by referencing week 2 of year 2024 docker-compose setup from FAQs database section "Project." You should find a link or reference to it in there which you could follow if needed. The exact details are not mentioned but this gives the direction that one already has access to such resources as per context.\n   \n2) In your local machine, create a directory `dbt` and navigate into: \n   ```shell\n   mkdir dbt && cd dbt\n   ```\n3) Clone the needed repository with git command by replacing `<your-path>` in this line from FAQs database section "Project":\n   ```shell\n   vi dbt/profiles.yml  # Edit as required and then save it using Ctrl + X or right click -> Save"\n   \n4) Navigate to the `dbt` folder, create a project directory inside of it:\n   ```shell0 bash -c "cd ../project && mv dbt-starter-projec

In [14]:
print(_)

 Yes, you can run dbt locally using Docker and Postgres as per the provided context. Here are detailed steps based on that information:

1) First, ensure your postgres instance is running by referencing week 2 of year 2024 docker-compose setup from FAQs database section "Project." You should find a link or reference to it in there which you could follow if needed. The exact details are not mentioned but this gives the direction that one already has access to such resources as per context.
   
2) In your local machine, create a directory `dbt` and navigate into: 
   ```shell
   mkdir dbt && cd dbt
   ```
3) Clone the needed repository with git command by replacing `<your-path>` in this line from FAQs database section "Project":
   ```shell
   vi dbt/profiles.yml  # Edit as required and then save it using Ctrl + X or right click -> Save"
   
4) Navigate to the `dbt` folder, create a project directory inside of it:
   ```shell0 bash -c "cd ../project && mv dbt-starter-project/* ."
``` 
5)